<a href="https://colab.research.google.com/github/arjunjanamatti/learn_and_practise_spacy/blob/master/text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.5.4

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
     |████████████████████████████████| 215.7MB 68kB/s 
     |████████████████████████████████| 204kB 48.1MB/s 
     |████████████████████████████████| 133kB 2.9MB/s 


In [3]:
import sparknlp

spark = sparknlp.start()

print('Spark NLP version: ', sparknlp.version())

Spark NLP version:  2.5.4


### creating spark dataframe

In [6]:
sample_text = 'Aj is a nice boxer from London'

spark_df = spark.createDataFrame(data = [[sample_text]]).toDF('text')

spark_df.show(truncate = False)

+------------------------------+
|text                          |
+------------------------------+
|Aj is a nice boxer from London|
+------------------------------+



In [13]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt

In [14]:
with open('./sample-sentences-en.txt') as f:
  print (f.read())

Peter is a very good person.
My life in Russia is very interesting.
John and Peter are brothers. However they don't support each other that much.
Lucas Nogal Dunbercker is no longer happy. He has a good car though.
Europe is very culture rich. There are huge churches! and big houses!


In [15]:
spark_df = spark.read.text(paths = './sample-sentences-en.txt').toDF('text')

spark_df.show(truncate = False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [16]:
spark_df.select('text').show(truncate = False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [19]:
text_files = spark.sparkContext.wholeTextFiles(path = './*.txt', 
                                               minPartitions = 4)

text_files

org.apache.spark.api.java.JavaPairRDD@7b1d4e8e

In [23]:
spark_df_folder = text_files.toDF(schema = ['path', 'text'])
spark_df_folder.show(truncate = False)

+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|path                                 |text                                                                                                                                                                                                                                                                                        |
+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|file:/content/sample-sen

### Document assembler

In [24]:
from sparknlp.base import *

In [25]:
documentAssembler = DocumentAssembler().setInputCol('text').setOutputCol('document').setCleanupMode('shrink')

doc_df = documentAssembler.transform(spark_df)

doc_df.show(truncate = False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|Peter is a very good person.                                                 |[[document, 0, 27, Peter is a very good person., [sentence -> 0], []]]                                                 |
|My life in Russia is very interesting.                                       |[[document, 0, 37, My life in Russia is very interesting., [sentence -> 0], []]]                                       |


In [26]:
doc_df.select('document.result', 'document.begin', 'document.end').show(truncate = False)

+-------------------------------------------------------------------------------+-----+----+
|result                                                                         |begin|end |
+-------------------------------------------------------------------------------+-----+----+
|[Peter is a very good person.]                                                 |[0]  |[27]|
|[My life in Russia is very interesting.]                                       |[0]  |[37]|
|[John and Peter are brothers. However they don't support each other that much.]|[0]  |[76]|
|[Lucas Nogal Dunbercker is no longer happy. He has a good car though.]         |[0]  |[67]|
|[Europe is very culture rich. There are huge churches! and big houses!]        |[0]  |[68]|
+-------------------------------------------------------------------------------+-----+----+



Flattening the document column can be done as following;

In [27]:
import pyspark.sql.functions as F

In [29]:
doc_df.withColumn(colName = 'tmp',
                  col = F.explode('document')).select('tmp.*').show(truncate = False)

+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|annotatorType|begin|end|result                                                                       |metadata       |embeddings|
+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|document     |0    |27 |Peter is a very good person.                                                 |[sentence -> 0]|[]        |
|document     |0    |37 |My life in Russia is very interesting.                                       |[sentence -> 0]|[]        |
|document     |0    |76 |John and Peter are brothers. However they don't support each other that much.|[sentence -> 0]|[]        |
|document     |0    |67 |Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[sentence -> 0]|[]        |
|document     |0    |68 |Europe is very culture rich. There are huge churches! and 

### Sentence Detector